In [52]:
question= "What is the biggest animal on earth"

document= "The most huge mammal in the world is elephant "

### A function that calculates the token for a given word

In [28]:
import tiktoken

def num_tokens_from_string (string:str,encoding_name:str)->int:
    '''Takes a token -> encodes it-> counts the number of tokens'''
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    print(encoding.encode(string))
    return num_tokens

tokens = num_tokens_from_string(question,'cl100k_base')


[3923, 374, 279, 8706, 10065, 389, 9578]


In [53]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")


query_result = model.encode([question])
document_result = model.encode([document])

In [54]:
print(type(query_result))
print(type(query_result))

print(query_result.shape)
print(document_result.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1, 768)
(1, 768)


In [55]:
import numpy  as np

def cosine_similarity(vect1,vect2):
    vect1 = vect1.flatten()  # convert (1, 384) → (384,)
    vect2 = vect2.flatten()
    dot_product = np.dot(vect1.flatten(),vect2)
    norm1 = np.linalg.norm(vect1)
    norm2 = np.linalg.norm(vect2)
    similarity = dot_product /  (norm1*norm2)
    return similarity

similarity_score = cosine_similarity(query_result, document_result)
print("Cosine similarity:", similarity_score)

Cosine similarity: 0.7984286


In [56]:
#load document
with open('documents.txt',"r",encoding="utf-8") as f:
    loaded_docs = [line.strip() for  line in f.readlines()]




In [57]:
for  elem in loaded_docs:
    print(" elem",elem)
  

 elem "The blue whale is known as the largest animal on Earth, reaching lengths of up to 100 feet and weights of around 200 tons. These marine mammals dominate the oceans in size and are remarkable creatures.",
 elem "Elephants are the largest land animals, with males weighing up to 12,000 pounds and standing 10-13 feet tall at the shoulder. Their massive size, intelligence, and social behavior make them truly impressive.",
 elem "Whales are among the largest living animals in the oceans. The blue whale holds the record for the biggest, surpassing all other creatures, both on land and in the sea, in sheer mass and length.",
 elem "Sharks vary in size, but the whale shark is the biggest fish in the world, reaching lengths of over 60 feet. Despite their enormous size, whale sharks are gentle filter feeders.",
 elem "The largest creatures ever to exist are marine animals, specifically whales. Their immense size allows them to store energy efficiently and navigate vast oceanic distances du

In [59]:
import numpy as np
encoded_documents = np.zeros((5,768))

for i,doc in enumerate(loaded_docs):
    encoded_documents[i] =model.encode([loaded_docs[i]])


In [60]:
for document in encoded_documents:
    print(cosine_similarity(document,query_result))

0.7713140596920469
0.7027656968877771
0.6811890522403985
0.5588066512575273
0.5971161464954607


## build a text splitter for pdf

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("hungary_immigration.pdf")
docs = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =1000,
    chunk_overlap=200,
    length_function= len
)

splits = text_splitter.split_documents(docs)

## Building vectorstore and retriever

In [133]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents= splits,embedding=embedding)
retriever = vectorstore.as_retriever(search_kwargs ={"k":2})

## Building the rag chain

### Creating the tamplate

In [ ]:
from langchain_core.prompts import PromptTemplate

# Define the RAC prompt
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Use the following context to answer the question accurately.

Context:
{context}

Question:
{question}

Answer:
"""
)


### Setting up the  DeepSeek LLM

In [145]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

API_KEY = os.getenv("DEEP_SEEK_API_KEY")
if not API_KEY:
    raise ValueError("DEEP_SEEK_API_KEY not found in environment variables")

# Use OpenAI-compatible client directly
llm = ChatOpenAI(
    api_key=API_KEY,
    base_url="https://api.deepseek.com/v1",
    model="deepseek-chat"
)

### Building the RAG chain

In [148]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Invoke
question = " I want to learn about family reunification"
result = rag_chain.invoke(question)
print(result)

Based on the provided legal text from Hungary's immigration law, here is what you can learn about family reunification:

**Eligible Family Members for Reunification:**
A residence permit for the purpose of family coexistence can be issued to:
*   The **dependent parent** of a family reunifier, their spouse, or a recognized refugee.
*   The **brother and direct relative** of the above, but only if the relative is unable to take care of themselves due to their health condition.

**Specific Conditions for Spouses:**
*   The **spouse of a recognized refugee** is eligible if the marriage was concluded *before* the refugee entered Hungary.
*   The **spouse of a family reunifier** is *not* eligible if the reunifier's *other spouse* already has a residence visa or permit for family coexistence (this clause appears to address polygamous marriages).

**Important Procedural Note:**
An application for a residence permit for the purpose of family reunification **cannot be refused solely** because a